In [1]:
import numpy as np
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia


In [2]:
d = pd.read_csv("data/planets.csv")

# Print the first few rows of the DataFrame
print(d.info())

Gaia.ROW_LIMIT = 10000

# print the row with the planet name: "Earth"

planet = d[d["pl_name"] == "eps Eri b"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5514 entries, 0 to 5513
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pl_name         5514 non-null   object 
 1   hostname        5514 non-null   object 
 2   sy_snum         5514 non-null   int64  
 3   sy_pnum         5514 non-null   int64  
 4   disc_facility   5514 non-null   object 
 5   disc_telescope  5514 non-null   object 
 6   pl_orbper       5271 non-null   float64
 7   pl_orbsmax      5217 non-null   float64
 8   pl_rade         5495 non-null   float64
 9   pl_radj         5494 non-null   float64
 10  pl_bmasse       5488 non-null   float64
 11  pl_bmassj       5488 non-null   float64
 12  pl_bmassprov    5514 non-null   object 
 13  pl_dens         5402 non-null   float64
 14  pl_orbeccen     4749 non-null   float64
 15  pl_insol        3834 non-null   float64
 16  pl_eqt          4072 non-null   float64
 17  pl_orbincl      4224 non-null   f

In [3]:
target_ra = planet["ra"].values[0]
target_dec = planet["dec"].values[0]
target_distance = planet["sy_dist"].values[0]

target_coord = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg, distance=target_distance * u.pc, frame='icrs')

target_ra = target_coord.ra.deg  # Right Ascension in degrees
target_dec = target_coord.dec.deg  # Declination in degrees
target_distance = target_coord.distance.pc # Distance in parsecs

print(target_coord.cartesian.xyz.value)

print(target_coord)

[ 1.89109805  2.53049979 -0.52627534]
<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (53.2284306, -9.4581715, 3.2026)>


In [4]:
def gen_query(target_ra, target_dec, target_distance, search_radius, search_distance, max_mag, TOP=30000):
    return f"""
        SELECT TOP {TOP}
        source_id, ra, dec, parallax, pmra, pmdec, phot_g_mean_mag
        FROM gaiadr3.gaia_source
        WHERE
        parallax IS NOT NULL
        AND ABS(1000/parallax - {target_distance}) < {search_distance}
        AND 1=CONTAINS(
            POINT('ICRS',ra,dec),
            CIRCLE('ICRS',{target_ra},{target_dec},{search_radius})
        )
        AND phot_g_mean_mag < {max_mag}
        ORDER BY distance_gspphot DESC 
    """

In [5]:
from astropy.coordinates import Distance

search_distance_o = 100  # distance in parsecs around the target planet
search_distance = min(search_distance_o, target_distance)



search_radius = np.arcsin(search_distance / np.sqrt((target_distance - search_distance) ** 2 + search_distance ** 2)) * 180 / np.pi
max_mag = 12

query = []

if search_distance_o > target_distance:
  query.append(gen_query(target_ra, target_dec, target_distance, 90, search_distance_o + target_distance, max_mag, 15000))
  query.append(gen_query((target_ra+180)%360, (target_dec+45)%90, search_distance_o - target_distance, search_radius, search_distance, max_mag, 15000))
else:
  query.append(gen_query(target_ra, target_dec, target_distance, search_radius, search_distance_o, max_mag))

data_df = None
for q in query:
  # print(q)
  job = Gaia.launch_job_async(q)
  raw_data = job.get_results()
  if data_df is None:
    data_df = raw_data.to_pandas()
  else:
    flines = len(data_df.index)
    new_data_df = raw_data.to_pandas()
    data_df = pd.concat([data_df, new_data_df], axis=0)
    print(f"{flines} + {len(new_data_df.index)} = {len(data_df.index)}")


data_df["parallax"] = data_df.apply(lambda row: abs(row["parallax"]), axis=1)  # convert parallax to parsecs
data_df["dist_earth"] = data_df.apply(lambda row: 1000 / row["parallax"],
                                      axis=1)  # calculate the distance from the parallax

target_coord = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg, distance=target_distance * u.pc, frame='icrs')
target_x, target_y, target_z = target_coord.cartesian.xyz.value

# calculate the cartesian coordinates with the target planet as the origin
star_spheric = SkyCoord(
    ra=data_df["ra"].values * u.deg,
    dec=data_df["dec"].values * u.deg,
    distance=data_df["dist_earth"].values * u.pc,
    frame='icrs'
)
data_df["x"] = data_df["dist_earth"] * np.cos(data_df["dec"]) * np.cos(data_df["ra"] - target_ra)
data_df["y"] = data_df["dist_earth"] * np.cos(data_df["dec"]) * np.sin(data_df["ra"] - target_ra)
data_df["z"] = data_df["dist_earth"] * np.sin(data_df["dec"])

# calculate the spherical coordinates with the target planet as the origin
star_cartesian = SkyCoord(
    x=data_df["x"].values - target_x,
    y=data_df["y"].values - target_y,
    z=data_df["z"].values - target_z,
    unit='pc',
    representation_type='cartesian'
)
data_df["ra_star"] = star_cartesian.spherical.lon.value
data_df["dec_star"] = star_cartesian.spherical.lat.value
data_df["dist_star"] = star_cartesian.spherical.distance.value

distance = Distance(parallax=data_df['parallax'].values * u.mas)

# Add the absolute magnitude to the dataframe
data_df['absolute_magnitude'] = data_df['phot_g_mean_mag'] - 5 * np.log10(distance.pc) + 5

data_df['relative_magnitude'] = data_df['absolute_magnitude'] + 5 * np.log10(data_df['dist_star']) - 5

data_df = data_df.query(f"dist_star < {search_distance_o}")  # filter out stars that are too far away
display(data_df)


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
15000 + 4271 = 19271


,SOURCE_ID,ra,dec,parallax,pmra,pmdec,phot_g_mean_mag,dist_earth,x,y,z,ra_star,dec_star,dist_star,absolute_magnitude,relative_magnitude
0,3303393870027685120,61.937192,9.756052,19.403370,68.822913,-18.051372,11.960291,51.537438,36.767193,-31.989088,-16.762436,315.294343,-18.307939,51.687105,8.399677,11.966588
1,943411659883480576,99.073156,37.991415,10.345682,59.533464,-133.114921,8.209749,96.658679,-26.612067,88.650467,27.853059,108.313018,17.371932,95.049807,3.283545,8.173301
2,2649243653327170304,341.665496,-3.526339,17.095480,-48.262209,-95.660741,11.416371,58.494995,-45.068778,30.140763,21.954591,149.546416,22.424966,58.931706,7.580778,11.432523
3,2425675006249455744,8.907769,-10.072100,33.792767,-187.403049,-68.151740,10.872495,29.592131,-22.267214,7.835777,17.845615,167.614191,36.604198,30.810654,8.516613,10.960118
5,5530317851619425024,129.841178,-36.606435,24.409283,-169.736083,52.890801,6.185167,40.968020,6.574495,17.666104,36.373939,72.806409,66.763044,40.157771,3.122942,6.141790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265,3604599342378116864,202.416153,-16.555125,10.535817,-60.629600,3.599413,11.057120,94.914332,2.382985,62.798766,71.129225,89.532384,49.932419,93.632348,6.170461,11.027591
4266,4271928438590799104,278.958213,-1.004121,10.675646,22.058223,-39.735778,11.254492,93.671143,44.948760,-22.544156,-79.029444,329.785592,-57.596316,92.980902,6.396463,11.238431
4267,4165665694951126272,266.791839,-8.072880,10.235824,67.493270,-19.832044,8.630053,97.696094,-21.170527,1.375736,-95.364788,182.866574,-76.316321,97.608993,3.680667,8.628116
4269,2282619025830656000,341.394183,78.903174,10.131240,76.705919,61.863981,8.922217,98.704601,-60.139973,69.965008,-35.081042,132.610063,-20.662948,97.924964,3.950530,8.904997


In [6]:
import plotly.graph_objs as go

print(target_x, target_y, target_z)

max_range = max(
    np.max(data_df["x"].values) - target_x, np.max(data_df["y"].values) - target_y,
    np.max(data_df["z"].values) - target_z,
    target_x - np.min(data_df["x"].values), target_y - np.min(data_df["y"].values),
    target_z - np.min(data_df["z"].values)
)

# Set limits for each axis centered on the target
x_limits = [target_x - max_range, target_x + max_range]
y_limits = [target_y - max_range, target_y + max_range]
z_limits = [target_z - max_range, target_z + max_range]

# Assuming `data_df` contains your star data along with relative magnitudes
# Let's say you have a 'relative_magnitude' column in the `data_df` DataFrame

# Normalize the relative magnitudes for color mapping
min_magnitude = data_df['relative_magnitude'].min()
max_magnitude = data_df['relative_magnitude'].max()

# Scale the relative magnitudes to a range [0, 1] for color interpolation
normalized_magnitude = (data_df['relative_magnitude'] - min_magnitude) / (max_magnitude - min_magnitude)

# Create a custom color array transitioning from white to blue
# Define white (1, 1, 1) and blue (0, 0, 1) in RGB format
colors = [
    f'rgba({int(255 * (1 - norm))},{int(255 * (1 - norm))},{255}, {255 * (1 - norm)})' for norm in normalized_magnitude
]

# Create the central target trace
target_trace = go.Scatter3d(
    x=[target_x], y=[target_y], z=[target_z],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Central Target'
)

# Create the Earth trace
earth_trace = go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers',
    marker=dict(size=10, color='green'),
    name='Earth'
)

# Create the star cloud trace with the color array
star_trace = go.Scatter3d(
    x=data_df["x"].values, y=data_df["y"].values, z=data_df["z"].values,
    mode='markers',
    marker=dict(size=2, color=colors),  # Use custom colors for stars
    name='Stars'
)

print('count of stars:', len(data_df))

# Define layout with manually set axis limits for equal scaling
layout = go.Layout(
    title='3D Star Field Around Target (Equal Scaling)',
    scene=dict(
        xaxis=dict(title='X (parsecs)', range=x_limits),
        yaxis=dict(title='Y (parsecs)', range=y_limits),
        zaxis=dict(title='Z (parsecs)', range=z_limits),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)  # Enforces equal scaling on all axes
    ),
)

# Create the figure
fig = go.Figure(data=[target_trace, star_trace, earth_trace], layout=layout)

# Show the plot
fig.show()

1.8910980539821243 2.5304997870073214 -0.5262753444547117
count of stars: 14307
